#

# LangGraph Jira without the Evaluator
This example exands on the second example and will have a similar flow except for the last agent/node that will call a new agent that will load the data to Jira using a LangChain tool.  Overall, this example will demonstrate a BSA agent that creates requirements, and evaluator to make sure they are complete, and then a recorder node that will enter them into Jira.

### 1. Load the needed libraries and environment variables

In [ ]:
from datetime import datetime
from dotenv import load_dotenv
from typing import List, Any, Optional, Annotated, TypedDict
from pydantic import BaseModel, Field

import os
import sqlite3
import gradio 

from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, ToolMessage
from langchain_core.tools import BaseTool
from langsmith import uuid7
from langchain_community.tools import Tool
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper
from langchain_community.tools.wikipedia.tool import WikipediaQueryRun
from langchain_community.agent_toolkits.jira.toolkit import JiraToolkit
from langchain_community.utilities.jira import JiraAPIWrapper
from langchain_community.agent_toolkits import FileManagementToolkit
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.prebuilt import ToolNode
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from IPython.display import Image

In [ ]:
load_dotenv(override=True)

### 2. Add Tools

In [ ]:
def get_tools() -> List[BaseTool]:
    # Define server, wrapper around the Google API, as a tool
    tool_search: Tool = Tool(
        name="search",
        func=GoogleSerperAPIWrapper().run,
        description="Use this tool when you want to get the results of an online web search"
    )

    # Define Wikipedia as a search as a tool
    wikipedia: WikipediaAPIWrapper = WikipediaAPIWrapper()
    wiki_tool: WikipediaQueryRun = WikipediaQueryRun(api_wrapper=wikipedia)

    # Define file management tools
    file_tools: List[BaseTool] = FileManagementToolkit(root_dir="sandbox").get_tools()

    # Concatenate all tools together as a list
    tools: List[BaseTool] = file_tools + [tool_search, wiki_tool]

    return tools

### 2a. Add Tools for the Recorder agent

In [ ]:
def get_recorder_tools() -> List[BaseTool]:
    import json
    from langchain_core.tools import StructuredTool
    
    # Define file management tools
    file_tools: List[BaseTool] = FileManagementToolkit(root_dir="sandbox").get_tools()

    # Define Jira tools with explicit configuration
    jira_instance_url = os.environ.get("JIRA_INSTANCE_URL")
    jira_username = os.environ.get("JIRA_USERNAME")
    jira_api_token = os.environ.get("JIRA_API_TOKEN")
    
    if not all([jira_instance_url, jira_username, jira_api_token]):
        raise ValueError("Missing required Jira environment variables: JIRA_INSTANCE_URL, JIRA_USERNAME, or JIRA_API_TOKEN")
    
    # Initialize JiraAPIWrapper with explicit configuration
    jira_api = JiraAPIWrapper(
        jira_instance_url=jira_instance_url,
        jira_username=jira_username,
        jira_api_token=jira_api_token
    )
    
    jira_toolkit = JiraToolkit.from_jira_api_wrapper(jira_api)
    jira_tools: List[BaseTool] = jira_toolkit.get_tools()
    
    # Create a custom create_issue tool that handles dict inputs
    def create_issue_wrapper(issue_data) -> str:
        """
        Create a Jira issue. The issue_data should be a JSON string with fields like:
        {"summary": "...", "description": "...", "issuetype": "Story", "project": "AUT"}
        """
        try:
            # Handle LangChain's wrapped argument format
            if isinstance(issue_data, dict):
                # Check if this is a LangChain wrapped argument
                if '__arg1' in issue_data:
                    issue_data = issue_data['__arg1']
                else:
                    issue_data = json.dumps(issue_data)
            
            # Validate that we have a non-empty string
            if not issue_data or not isinstance(issue_data, str):
                return f"Error: Invalid issue_data. Received: {type(issue_data)} - {repr(issue_data)}"
            
            # Try to parse as JSON to validate format
            parsed = json.loads(issue_data)
            if not isinstance(parsed, dict):
                return "Error: issue_data must be a JSON object (dict)"
            
            # Convert simple format to Jira API format
            # LLM provides: {"summary": "...", "description": "...", "issuetype": "Story", "project": "AUT"}
            # Jira API expects: {"summary": "...", "description": "...", "issuetype": {"name": "Story"}, "project": {"key": "AUT"}}
            
            jira_format = {
                "summary": parsed.get("summary"),
                "description": parsed.get("description"),
                "issuetype": {"name": parsed.get("issuetype", "Story")},
                "project": {"key": parsed.get("project", "AUT")}
            }
            
            # Validate required fields
            if not jira_format.get("summary"):
                return "Error: 'summary' field is required"
            if not jira_format.get("description"):
                return "Error: 'description' field is required"
            
            # Call the direct Jira create_issue method with the properly formatted data
            result = jira_api.jira.create_issue(fields=jira_format)
            return f"Successfully created issue: {result.key} ({result.fields.summary})"
            
        except json.JSONDecodeError as e:
            return f"Error: Invalid JSON format in issue_data: {str(e)}\nReceived: {repr(issue_data)}"
        except Exception as e:
            return f"Error creating issue: {str(e)}"
    
    # Replace the create_issue tool with our wrapper
    custom_create_issue_tool = Tool(
        name="create_issue",
        func=create_issue_wrapper,
        description="Create a Jira issue. Pass a JSON string with fields: summary (required), description (required), issuetype (required, use 'Story'), and project (required, use 'AUT'). Example: '{\"summary\": \"Title\", \"description\": \"Details\", \"issuetype\": \"Story\", \"project\": \"AUT\"}'"
    )
    
    # Add a custom tool to list available Jira fields using StructuredTool for zero-argument tool
    def list_jira_fields() -> str:
        """List all available Jira fields that can be used when creating issues in the AUT project."""
        try:
            # Access the underlying Jira client to get field information
            fields = jira_api.jira.fields()
            available_fields = []
            for field in fields:
                available_fields.append({
                    "id": field.get("id"),
                    "name": field.get("name"),
                    "schema": field.get("schema", {}).get("type", "unknown")
                })
            
            # Format the output to be readable
            field_list = "Available Jira Fields:\n"
            for field in available_fields:
                field_list += f"- {field['name']} (ID: {field['id']}, Type: {field['schema']})\n"
            return field_list
        except Exception as e:
            return f"Error retrieving fields: {str(e)}"
    
    fields_tool = StructuredTool.from_function(
        func=list_jira_fields,
        name="list_jira_fields",
        description="List all available Jira fields that can be used when creating issues in the AUT project. Call this with no arguments to discover field options."
    )
    
    # Get all Jira tools except create_issue, then add our custom ones
    other_jira_tools = [t for t in jira_tools if t.name != "create_issue"]
    
    # Concatenate all tools together as a list
    tools: List[BaseTool] = file_tools + other_jira_tools + [custom_create_issue_tool, fields_tool]
    return tools

### 3. Setup Memory

In [ ]:
db_path = "memory.db"
conn = sqlite3.connect(db_path, check_same_thread=False)
sql_memory = SqliteSaver(conn)

### 4. Build the Graph
It is now time to put start building the graph.  All graphs include the following steps:
<ol>
<li>Define the State</li>
<li>Start Graph Builder</li>
<li>Create Nodes</li>
<li>Create Edges</li>
<li>Compile the Graph</li>
</ol>

### State with additional details
Will track additional details in the state to help with routing:
<ul>
<li>file_name: name of the file</li>
<li>feedback_on_work: details from the evaluator</li>
<li>success_criteria_met: determines if it should end</li>
<li>number_of_reviews: the number of reviews completed as a guardrail from looping too much</li>
</ul>

In [ ]:
# Step 1: Define the State object
class State(TypedDict):
    messages: Annotated[List[Any], add_messages]
    file_name: str
    feedback_on_work: Optional[str]
    success_criteria_met: bool
    number_of_reviews: int=0

In [ ]:
# Step 2: Start the Graph Builder with this State class
graph_builder = StateGraph(State)

#### Define the prompts

In [ ]:
current_time = datetime.now().strftime("%Y-%m-%d %H:%M")

bsa_system_message = f"""
You are a Business Systems Analyst (BSA), an autonomous agent specializing in transforming a short requirement into a complete set of functional and non-functional requirements, user stories, and acceptance criteria. You have access to tools including Google Search, Wikipedia, and file management tools (read_file, write_file, list_directory, delete_file).

Your responsibilities include:
1. Analyzing the user-provided requirement.
2. DO NOT ask clarifying questions.  Make educated assumptions.
3. Extracting and defining functional and non-functional requirements.
4. Converting each requirement into one or more user stories.
5. Writing acceptance criteria for every user story.
6. Compiling all output into a properly structured markdown file.
7. Saving the final markdown file to disk using the write_file tool.

TOOL USAGE RULES:
- You MUST NOT call write_file until the entire markdown document is fully generated.
- Only use search tools when information cannot be derived from the provided requirement.

OUTPUT REQUIREMENTS:

1. The final output MUST be a single markdown file with the following structure:

   # Overview  
   # Functional Requirements  
   # Non-Functional Requirements  
   # User Stories  
   ## <User Story Title>  
   - User Story  
   - Acceptance Criteria  

2. Each User Story MUST follow the exact pattern:
       As a <type of user>,
       I want <goal or action>,
       So that <reason or value>.

3. Each requirement must produce at least one user story.  
   If a requirement implies multiple user needs, generate multiple user stories.

4. Each User Story MUST include **at least one** Acceptance Criteria, and as many as needed for full clarity.

5. All Acceptance Criteria MUST follow the exact Given/When/Then pattern:
       Given <precondition>
       When <action>
       Then <expected outcome>

6. Tone rules:
   - Use a neutral, authoritative, academic tone for descriptions, lists, analyses, and requirement sections.
   - User stories themselves should retain standard conversational user-story tone.

FILENAME REQUIREMENTS (MANDATORY):

You MUST save the final markdown file using the write_file tool.

The filename MUST follow this exact format:
       YYYYMMDDHHmm_<requirement_summary>.md

Where:
- YYYYMMDDHHmm is the current UTC timestamp (year, month, day, hour, minute, second).
- The current date and time is: {current_time}
- <requirement_summary> is a short, snake_case summary of the requirement:  
      - all lowercase  
      - alphanumeric plus underscores  
      - no spaces  
      - concise (ideally 3–6 words)


BEHAVIOR RULES:
- NEVER skip functional or non-functional requirements.
- ALWAYS ensure full coverage: no part of the requirement should remain unmapped to a story.
- NEVER call write_file until the markdown document is 100% complete.
- WRITE ALL requirements to a single file.
- ALWAYS confirm the file has been saved after using write_file.

FINAL WORKFLOW:
1. Parse the user requirement.
2. Generate functional requirements.
3. Generate non-functional requirements.
4. Convert all requirements into user stories.
5. Define acceptance criteria for each story.
6. Compile the full markdown report.
7. Save to disk using write_file with the required filename format.
8. Confirm the file has been saved.

"""




In [ ]:
recorder_system_message = """
You are a Jira Recorder Agent responsible for converting user stories from a BSA requirements document into Jira issues.

YOUR TASK IS MANDATORY: You MUST create Jira issues for all user stories in the requirements document. This is not optional.

IMMEDIATE ACTIONS YOU MUST TAKE (in this order):
1. FIRST: Call the "list_jira_fields" tool to see what Jira fields are available AND to verify the correct project key
2. SECOND: Use the read_file tool to read the markdown requirements file from the sandbox directory
3. THIRD: Extract each user story and its acceptance criteria from the markdown
4. FOURTH: For EACH user story, call the create_issue tool to create a Jira issue

DO NOT generate any explanatory text. DO NOT summarize. DO NOT confirm without taking action.
You MUST call tools to complete your task.

REQUIRED TOOL CALLS:
1. list_jira_fields - Call this FIRST to discover the correct project key
2. read_file - Call this SECOND with the requirements markdown file
3. create_issue - Call this for EACH user story (mandatory)

IMPORTANT NOTE ABOUT PROJECT KEY:
The project key should be "AUT" but verify this by examining the output from list_jira_fields. 
If the fields list shows a different project key should be used, use that instead.
NEVER make assumptions about the project key - verify it first.

JIRA ISSUE CREATION - EXACT FORMAT REQUIRED:

When calling the create_issue tool, you MUST pass a valid JSON string. Here is the exact format:

{"summary": "User Story Title", "description": "[User Story]\\nAs a <type of user>,\\nI want <goal or action>,\\nSo that <reason or value>.\\n\\n[Acceptance Criteria]\\nGiven <precondition>\\nWhen <action>\\nThen <expected outcome>", "issuetype": "Story", "project": "AUT"}

Breaking it down:
- summary: The user story title (required)
- description: The full user story + acceptance criteria (required)
- issuetype: MUST be "Story" (required)
- project: MUST be a valid project key (required) - verify the correct key from list_jira_fields

EXAMPLES:

Example 1 - Simple user story:
{"summary": "Track vehicle purchase information", "description": "[User Story]\\nAs a car owner,\\nI want to record my vehicle purchase details,\\nSo that I have a complete history of my vehicle ownership.\\n\\n[Acceptance Criteria]\\nGiven that I have registration details to enter,\\nWhen I submit the information,\\nThen the registration should be recorded in my account.", "issuetype": "Story", "project": "AUT"}

CRITICAL RULES:
- Always pass create_issue a valid JSON string - check that all quotes are properly escaped
- NEVER use custom fields (customfield_XXXXX) 
- Each user story MUST become a separate Jira issue
- Include all acceptance criteria in the description
- Do NOT modify the user stories or acceptance criteria
- You MUST call create_issue for every user story in the document
- Use \\n for line breaks in the JSON string
- Verify the project key is correct before attempting to create issues

WORKFLOW:
1. Call list_jira_fields to see what fields are available
2. Call read_file to get the requirements file
3. Parse the user stories
4. For each story, call create_issue with a valid JSON string
5. Report which issues were successfully created

BEGIN NOW. Call the list_jira_fields tool first to verify the project key.
"""

### Create the BSA

In [ ]:
open_ai = ChatOpenAI(model="gpt-4o-mini")
tools_for_bsa = get_tools()
bsa_open_ai_with_tools = open_ai.bind_tools(tools_for_bsa)

def bsa(state: State):
    # Convert incoming messages to BaseMessage objects
    messages = []
    for msg in state["messages"]:
        if isinstance(msg, dict):
            if msg["role"] == "system":
                messages.append(SystemMessage(content=msg["content"]))
            elif msg["role"] == "user":
                messages.append(HumanMessage(content=msg["content"]))
            elif msg["role"] == "assistant":
                messages.append(AIMessage(content=msg.get("content", ""), tool_calls=msg.get("tool_calls")))
            elif msg["role"] == "tool":
                messages.append(ToolMessage(content=msg["content"], tool_call_id=msg.get("tool_call_id")))
        else:
            messages.append(msg)
    # Ensure system message is present
    has_system_message = any(isinstance(m, SystemMessage) for m in messages)
    if not has_system_message:
        messages.insert(0, SystemMessage(content=bsa_system_message))
    results = bsa_open_ai_with_tools.invoke(messages)
    return {"messages": [results]}


# Add bsa and tools_for_bsa nodes
graph_builder.add_node("bsa", bsa)
graph_builder.add_node("tools_for_bsa", ToolNode(tools=tools_for_bsa))

### Create a Recorder

In [ ]:
open_ai_for_recorder = ChatOpenAI(model="gpt-4o-mini")
tools_for_recorder = get_recorder_tools()
recorder_open_ai_with_tools = open_ai_for_recorder.bind_tools(tools_for_recorder)

def recorder(state: State):
    # Convert incoming messages to BaseMessage objects
    messages = []
    for msg in state["messages"]:
        if isinstance(msg, dict):
            if msg["role"] == "system":
                messages.append(SystemMessage(content=msg["content"]))
            elif msg["role"] == "user":
                messages.append(HumanMessage(content=msg["content"]))
            elif msg["role"] == "assistant":
                messages.append(AIMessage(content=msg.get("content", ""), tool_calls=msg.get("tool_calls")))
            elif msg["role"] == "tool":
                messages.append(ToolMessage(content=msg["content"], tool_call_id=msg.get("tool_call_id")))
        else:
            messages.append(msg)
    
    # Ensure system message is present
    has_system_message = any(isinstance(m, SystemMessage) for m in messages)
    if not has_system_message:
        messages.insert(0, SystemMessage(content=recorder_system_message))
    
    # Add filename context if available
    if state.get("file_name"):
        messages.append(HumanMessage(content=f"The requirements file you need to read is: {state['file_name']}"))
    
    results = recorder_open_ai_with_tools.invoke(messages)
    return {"messages": [results]}


# Add recorder and tools_for_recorder nodes
graph_builder.add_node("recorder", recorder)
graph_builder.add_node("tools_for_recorder", ToolNode(tools=tools_for_recorder))

### Define condition statements.
In this case, will not just use the out of the box tool router as we have multiple different tools calls and some more complex flows 

In [ ]:
MAX_NUMBER_OF_REVIEWS = 4

def route_bsa(state: State):
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools_for_bsa"
    return "recorder"

def route_recorder(state: State):
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools_for_recorder"
    return "END"


In [ ]:
# Step 4: Create the edges

# Entry point
graph_builder.add_edge(START, "bsa")

# From bsa: if needs tools, go to tools; otherwise go to evaluator
graph_builder.add_conditional_edges(
    "bsa", 
    route_bsa, 
    {"tools_for_bsa": "tools_for_bsa", "recorder": "recorder"}
)

# From tools_for_bsa: always back to bsa to process tool results
graph_builder.add_edge("tools_for_bsa", "bsa")


# From recorder: if needs tools, go to tools; otherwise go to END
graph_builder.add_conditional_edges(
    "recorder", 
    route_recorder, 
    {"tools_for_recorder": "tools_for_recorder", "END": END}
)

# From tools_for_recorder: always back to recorder to process tool results  
graph_builder.add_edge("tools_for_recorder", "recorder")


In [ ]:
# Step 5: Compile the graph
graph = graph_builder.compile(checkpointer=sql_memory)
display(Image(graph.get_graph().draw_mermaid_png()))

### 5. Test it Out
Use Gradio to create a front end that will allow us to test this out

In [ ]:
user_input = """
Create requirements for a website that will track automobile information.  
It should track from the time of purchase, track all mileage, fuel, insurance, registration, repairs and any other associated costs.
"""
# 1. **User Roles**: This site is intended for individual car owners as a personal site with only a handful of users.
# 2. **Data Entry**: Users will manually enter the data through appropriate screens.
# 3. **Reports**: Should just include a single report that list all information about a single vechicle.
# 4. **Mobile Access**: The site should be mobile friendly so that it can be displayed in a phone's browser
# 5. **Integration**: There are no integrations needed.
# 6. **User Notifications**: There should be no user notifications or alerts at this time.
# 7. **Vehicle Information**: The following should be tracked: make, model, year, color, VIN, license plate number, purchase date, purchase price, who it was bought from and when applicable, when it was sold, sold price, and who it was sold to.
# 8. **Cost Tracking**: Cost tracking should include: fuel, insurance, registration, repairs, car washes, accessories, and a miscellaneous categetory that can take in a user note.
# 9. **User Authentication**: The site will initially be available to everyone with full access
# 10. **Data Security**: There any no specific security requirements or considerations for protecting the data entered by the users.
# """


config = {"configurable": {"thread_id": uuid7()}}

with gradio.Blocks() as demo:
    chatbot = gradio.Chatbot(type="messages")
    user_input_box = gradio.Textbox(value=user_input, label="Message")
    submit_btn = gradio.Button("Send")

    def chat_handler(message, history):
        # Invoke the graph with the user message
        result = graph.invoke({"messages": [{"role": "user", "content": message}]}, config=config)
        last_message = result["messages"][-1]
        
        # Convert the last message to proper format
        if isinstance(last_message, str):
            assistant_message = {"role": "assistant", "content": last_message}
        elif isinstance(last_message, dict):
            assistant_message = last_message
        else:
            # Handle LangChain message objects
            assistant_message = {"role": "assistant", "content": last_message.content}
        
        # Return updated history with both user and assistant messages
        return history + [{"role": "user", "content": message}, assistant_message]

    submit_btn.click(chat_handler, [user_input_box, chatbot], chatbot)

demo.launch()